In [39]:
pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
import time
import re

In [2]:
# 발급받은 API 키
api_key = "tXsTu3r9EslMT0tkioVB"
api_secret = 'qXR7d0OGoT'

In [3]:
# API 호출 URL
url = "https://openapi.naver.com/v1/search/news.json"

In [4]:
# 검색할 키워드들
keywords = ["강도", "살인", "강간", "성폭행", "칼부림", "묻지마", "폭행", "음주운전"]

In [5]:
# API 호출 시 사용할 헤더
headers = {
    "X-Naver-Client-Id": api_key,
    "X-Naver-Client-Secret": api_secret
}

In [6]:
# 결과를 저장할 리스트
news_list = []
naver_news_list = []

In [7]:
# 키워드별로 검색 결과 가져오기
for keyword in keywords:
    params = {
        "query": keyword,
        "display": 20,  # 가져올 결과 개수
        "start": 1      # 시작 위치 (1부터)
    }
    
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        for item in data['items']:
            link = item['link']
            if link.startswith("https://n.news.naver.com"):
                try:
                    news_response = requests.get(link)
                    if news_response.status_code == 200:
                        soup = BeautifulSoup(news_response.content, "html.parser")
                        title_element = soup.find("h2", id="title_area") or soup.find("h2", class_="end_tit")
                        title = title_element.get_text() if title_element else ""

                        content_element = soup.find("div", id="newsct_article") or soup.find("div", id="articeBody")
                        content = content_element.get_text() if content_element else ""

                        news_dict = {
                            "keyword": keyword,
                            "title": title,
                            "content": content
                        }
                        
                        naver_news_list.append(news_dict)
                    else:
                        print(f"뉴스 링크 '{link}' 접속 실패")
                except Exception as e:
                    print(f"뉴스 링크 '{link}' 크롤링 실패:", e)
            time.sleep(0.2)  # 0.2초 쉬기
    else:
        print(f"검색어 '{keyword}'의 API 호출 실패")

In [8]:
naver_news_list

[{'keyword': '강도',
  'title': '기장군도시관리공단, 지방공기업 경영 평가 4년 연속 “우수”기관',
  'content': '\n\n\n\n\n\n부산시 기장군도시관리공단(이사장 김윤재)은 2022년 행정안전부 주관 지방공기업 경영 평가 결과 우수(‘나’) 등급을 달성해 4년 연속 우수 기관으로 선정됐다.기장군도시관리공단은 전국 지방공기업 381개 기관을 대상으로 실시한 2022년도 지방공기업 경영 평가 고객 만족도 조사에서 2009년 개청 이래 최고 득점(종합만족도 92.9점)을 기록해 고객 서비스 향상을 이뤘다. 재무 건전성도 대폭 강화해 대행사업비 절감률 등 주요 경영 성과 지표 평가에서 직전 5개년도 중 최고 득점을 올렸다.기장군도시관리공단 김윤재 이사장은 “이번 경영 평가 결과는 2022년 정부 정책 기조를 반영해 추진한 재무 건전성과 혁신 강화 노력의 결실이라 생각한다”며 “2023년도 또한 강도 높은 경영 개선 노력으로 기관 혁신을 가속화해 최우수 지방공기업으로 도약할 수 있도록 최선을 다하겠다”고 말했다.\n\n'},
 {'keyword': '강도',
  'title': '"당근 거래해요" 만나 1200만원 롤렉스 빼앗고 폭행한 30대',
  'content': '\n\n강도치사죄로 징역 12년 선고받고 출소한 지 5개월도 안돼 범행\n\n\n\n판결 관련 자료 이미지. 매일신문 DB중고 거래를 하자고 속여 고가의 시계를 빼앗고 폭행한 30대에게 1심에서 징역 7년이 선고됐다.23일 법조계에 따르면 대전지법 형사12부(나성훈 부장판사)는 강도상해 혐의로 기소된 A(35)씨에게 징역 7년을 선고하고 10년간 위치추적 전자장치(전자발찌) 부착을 명령했다.A시는 지난 1월 28일 오후 7시쯤 대전 서구 한 음식점에서 중고 거래 장터인 당근마켓을 통해 만난 B(46)씨가 잠시 자리를 비운 사이 테이블 위에 있던 시가 1천200만원 상당의 롤렉스 시계를 훔쳐 달아나려고 했다.뒤쫓아 나온 B씨에게 붙잡히자 얼굴과 몸을 여러 차례 폭행

In [9]:
geolocator = Nominatim(user_agent="myGeocoder")

In [10]:
# 뉴스에서 추출한 정보를 저장할 리스트
crime_location = []

In [11]:
# naver_news_list에서 뉴스 제목과 내용 추출
for news in naver_news_list:
    news_text = news["title"] + " " + news["content"]
    
    # 지역 키워드 추출
    location_keywords = re.findall(r"\b(서울|부산|대구|인천|광주|대전|울산|충남|천안|아산)\b", news_text)
    
    if location_keywords:
        for location_keyword in location_keywords:
            location = geolocator.geocode(location_keyword)
            if location:
                crime_info = {
                    "사건": news['keyword'],
                    "지역": location_keyword,
                    "위도": location.latitude,
                    "경도": location.longitude
                }
                crime_location.append(crime_info)
            
            else:
                print(f"키워드 '{keyword}'로 추정되는 사건의 지역 정보를 찾을 수 없음")

In [12]:
crime_location

[{'사건': '강도', '지역': '대전', '위도': 36.3497007, '경도': 127.3849016},
 {'사건': '강도', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '강도', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '강도', '지역': '인천', '위도': 37.456, '경도': 126.7052},
 {'사건': '강도', '지역': '충남', '위도': 36.6593, '경도': 126.6729},
 {'사건': '강도', '지역': '부산', '위도': 35.1799528, '경도': 129.0752365},
 {'사건': '강도', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126.9782914},
 {'사건': '살인', '지역': '서울', '위도': 37.5666791, '경도': 126

In [51]:
# 이전에 생성한 m 객체가 있다면, 그 객체를 사용하고 없다면 새로 생성
if 'm' not in globals():
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

In [52]:
# 맵 중심 위치 설정
map_center = [37.5666791, 126.9782914]
m = folium.Map(location=map_center, zoom_start=10)

In [53]:
# 아이콘 설정
icons = {
    '강도': 'exclamation-circle', '살인': 'skull-crossbones', '강간': 'exclamation-triangle', '성폭행': 'exclamation',
    '칼부림': 'crosshairs', '묻지마': 'eye-slash', '폭행': 'hand-rock', '음주운전': 'car-crash'
}

In [54]:
# 사건 정보를 기반으로 지도에 마커 추가
for crime_info in crime_location:
    icon = folium.Icon(icon=icons[crime_info['사건']], color=colors[crime_info['사건']])
    
    folium.Marker(
        location=[crime_info['위도'], crime_info['경도']],
        icon=icon,
        popup=f"{crime_info['사건']} 사건\n지역: {crime_info['지역']}"
    ).add_to(m)

In [55]:
# 지도를 HTML 파일로 저장
m.save("crime1_map.html")